# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
# Passo 1 - Entrar na Internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

navegador = webdriver.Firefox()

# Passo 2 - Pegar Cotação do dólar
# - Entrar no google
navegador.get("https://www.google.com/")
# - Pesquisar "Cotação do dólar"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - Pegar o Número que aparece no resultado do google
time.sleep(2)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

# Passo 3 - Pegar Cotação do euro
navegador.get("https://www.google.com/")
# - Pesquisar "Cotação do euro"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - Pegar o Número que aparece no resultado do google
time.sleep(2)
cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)


# Passo 4 - Pegar a Cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
# - Pegar cotação
cotacao_ouro = navegador.find_element_by_xpath(
    '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
# Passo 5 - Importar e atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# Atualizar a cotação
# Aonde a coluna "Moeda" = "Dólar"
# tabela.loc[linha, coluna]
tabela.loc[tabela['Moeda'] == "Dólar", 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == "Euro", 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == "Ouro", 'Cotação'] = float(cotacao_ouro)

# Atualizar o Preço de Compra -> Preço Original * Cotação
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

# Atualizar o Preço de venda -> Preço de compra * Margem
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

tabela["Preço Final"] = tabela["Preço Final"].map("{:.2f}".format)

display(tabela)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
# Passo 6 - Exportar a base de dados atualizada
tabela.to_excel("Produtos Novo.xlsx", index=False)